In [3]:
using Pkg
using Revise
Pkg.activate(dirname(dirname(@__DIR__)))
using MacroEnergy
using Gurobi
using Plots
using DataFrames, CSV
using BenchmarkTools
using JuMP

  Activating project at `~/Documents/MacroEnergy.jl`


In [2]:
import Pkg; Pkg.add("Plots")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed xkbcommon_jll ──────────── v1.9.2+0
   Installed Scratch ────────────────── v1.3.0
   Installed libfdk_aac_jll ─────────── v2.0.4+0
   Installed Opus_jll ───────────────── v1.5.2+0
   Installed libpng_jll ─────────────── v1.6.50+0
   Installed GR_jll ─────────────────── v0.73.17+0
   Installed Plots ──────────────────── v1.40.16
   Installed Xorg_xcb_util_cursor_jll ─ v0.1.5+0
   Installed Wayland_jll ────────────── v1.23.1+1
   Installed LAME_jll ───────────────── v3.100.3+0
   Installed Qt6Wayland_jll ─────────── v6.8.2+1
   Installed Ogg_jll ────────────────── v1.3.6+0
   Installed Glib_jll ───────────────── v2.84.3+0
   Installed GettextRuntime_jll ─────── v0.22.4+0
   Installed GR ─────────────────────── v0.73.17
      Compat entries added for Plots
    Updating `~/Documents/MacroEnergy.jl/Project.toml`
  [91a5bcdd] + Plots v1.40.16
    Updating `~/Documents/MacroEnergy.jl/Man

In [4]:
include("helper_functions.jl");

In [5]:
system = MacroEnergy.load_system(@__DIR__)
model = MacroEnergy.generate_model(system)

┌ Info: Loading system from /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/eastern_us_three_zones_with_cement/system_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system.jl:39
┌ Info: Loading system data
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:17
┌ Info: Done loading system data. It took 0.31 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:25
┌ Info: Generating system
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/generate_system.jl:18
┌ Warning: Using default weights = 1 as no period map provided and each period maps to itself
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:198
┌ Warning: Using default weights = 1 as no period map provided and each period maps to itself
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:198
┌ Warning: Us

MethodError: MethodError: no method matching generate_model(::MacroEnergy.System)
The function `generate_model` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  generate_model(!Matched::MacroEnergy.Case)
   @ MacroEnergy ~/Documents/MacroEnergy.jl/src/model/generate_model.jl:2


In [3]:
MacroEnergy.set_optimizer(model, Gurobi.Optimizer);
MacroEnergy.set_optimizer_attributes(model, "BarConvTol"=>1e-3,"Crossover" => 0, "Method" => 2)
MacroEnergy.optimize!(model)

Set parameter Username
Set parameter LicenseID to value 2637694
Academic license - for non-commercial use only - expires 2026-03-17
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Non-default parameters:
Method  2
BarConvTol  0.001
Crossover  0

Optimize a model with 1147587 rows, 858532 columns and 3608083 nonzeros
Model fingerprint: 0x5ce34670
Coefficient statistics:
  Matrix range     [4e-07, 2e+04]
  Objective range  [1e-01, 4e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+04]
Presolve removed 640041 rows and 452795 columns
Presolve time: 0.72s
Presolved: 507546 rows, 405737 columns, 2406303 nonzeros
Ordering ti

In [6]:
case = "0pct_co2_reduction"
results_dir = joinpath(@__DIR__,  "results")
mkpath(results_dir)
save_results(results_dir, case, system)

"/Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/eastern_us_three_zones_with_cement/results/0pct_co2_reduction_flows.csv"

In [ ]:
# Run model for different CO2 caps, if you have a no_cap_co2_level

no_cap_co2_level = 1.602e6
co2_reductions = [0.2, 0.4, 0.6, 0.8] # If there is a 0, it must be the first value
system = nothing
model = nothing

for co2_reduction in co2_reductions
    # Load system
    case_path = @__DIR__
    println("###### ###### ######")
    println("Running case at $(case_path)")
    system = MacroEnergy.load_system(case_path)

    # Get the CO2 node
    nodes = Node[x for x in system.locations if x isa MacroEnergy.Node]
    co2_node = MacroEnergy.get_nodes_sametype(nodes, CO2)[1] # the cement_co2_sink node is the first CO2 node

    if co2_reduction == 0 # If there is no CO2 cap
        case = "0pct_co2_reduction"
        co2_node.constraints = AbstractTypeConstraint[] # Remove CO2 cap constraint
    else # If there is not a CO2 cap
        case = string(floor(Int, co2_reduction * 100)) * "pct_co2_reduction"
        co2_node.rhs_policy[CO2CapConstraint] = no_cap_co2_level * (1 - co2_reduction) # Set CO2 cap
        print(no_cap_co2_level * (1 - co2_reduction))
    end

    # Run model
    model = MacroEnergy.generate_model(system)
    MacroEnergy.set_optimizer(model, Gurobi.Optimizer);
    MacroEnergy.set_optimizer_attributes(model, "BarConvTol"=>1e-3,"Crossover" => 0, "Method" => 2)
    MacroEnergy.optimize!(model)

    # Save results
    results_dir = joinpath(case_path, "results")
    mkpath(results_dir)
    save_results(results_dir, case, system)
    # @save case * "_results.jld2" system_results_df capacity_results_df results_8760_df
    # println("wrote results")

    # Record the no_cap_co2_level for next iterations
    if co2_reduction == 0
        no_cap_co2_level = MacroEnergy.value(sum(co2_node.operation_expr[:emissions]))
    end
end